In [ ]:
import os
import omnikeeper_client as okc

In [ ]:
from dotenv import load_dotenv
load_dotenv('../.env.sample')

In [ ]:
trait_name = "python_client_demo.test"
layer_name = "testlayer"

create a omnikeeper client

In [ ]:
okapiclient = okc.OkApiClient(
    backend_url=os.getenv('OMNIKEEPER_URL'),
    client_id=os.getenv('OMNIKEEPER_AUTH_CLIENTID'),
    username=os.getenv('OMNIKEEPER_AUTH_USERNAME'),
    password=os.getenv('OMNIKEEPER_AUTH_PASSWORD'),
)

Create the input data in format of **input : List[Dict[str,Any]]**

In [ ]:
# df_init = pd.DataFrame.from_records([
#     {"id": 1, "array": ["a", "b"], "some_key": "Value 1"},
#     {"id": 3, "array": ["c", "d"], "some_key": "another Value"},
# ])

# df_init

initial_data = [
    {"id": 1, "array": ["a", "b"], "some_key": "Value 1"},
    {"id": 3, "array": ["c", "d"], "some_key": "another Value"},
]
initial_data

write data to layer ``testlayer``

In [ ]:
# simple_traits.bulk_replace(client, trait_name=trait_name, input=df_init, id_attributes=["id"], id_relations=[], write_layer=layer_name, filter={})

okc.bulk_replace_trait_entities_by_filter(okapiclient, trait_name=trait_name, input=initial_data, id_attributes=["id"], id_relations=[], write_layer=layer_name, filter={})

load data again from omnikeeper

In [ ]:
okapiclient.get_schema()

In [ ]:
c = okapiclient._get_graphql_client()
print(c.schema)
from gql import gql
with c as session:
    print(c.schema)
schema = c.execute(gql('''
  {
    __schema {
      types {
        name
      }
    }
  }
'''))
print(schema)

In [ ]:
# df_work = simple_traits.get_all(client, trait_name=trait_name, layers=[layer_name])
# df_work

updated_data = okc.get_all_traitentities(okapiclient, trait_name=trait_name, layers=[layer_name])
updated_data

modify the fetched traitentities

In [ ]:
# change things in the returned data
# change single value of existing data
updated_data[0]["array"] = ["a", "b", "z"]

# add a new row, using our own created CIID
updated_data.append({"id": 4, "array": ["x"], "some_key": "the new guy"})

# drop a row
updated_data.pop(1)

write it back

In [ ]:
# simple_traits.set_all(client, trait_name=trait_name, input=df_work, write_layer=layer_name)

okc.bulk_replace_trait_entities(okapiclient, trait_name=trait_name, input=updated_data, write_layer=layer_name)

Fetch again the data

In [ ]:
# df_final = simple_traits.get_all(client, trait_name=trait_name, layers=[layer_name])

final_data = okc.get_all_traitentities(okapiclient, trait_name=trait_name, layers=[layer_name])